<a href="https://colab.research.google.com/github/jorgemunozl/vla-test/blob/main/tests/train_val_pi05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Eleven Test

This test run the training using the same architecture that the scrip that we use for train seriosuly our models.

Cloning into 'XHUMAN'...
remote: Enumerating objects: 2084, done.
remote: Counting objects: 100% (290/290), done.
remote: Compressing objects: 100% (185/185), done.
remote: Total 2084 (delta 185), reused 184 (delta 102), pack-reused 1794 (from 2)
Receiving objects: 100% (2084/2084), 7.81 MiB | 8.89 MiB/s, done.
Resolving deltas: 100% (1328/1328), done.


In [2]:
%cd XHUMAN

/content/XHUMAN


In [3]:
!uv pip install -e .[pi]

Using Python 3.12.12 environment at: /usr
Resolved 215 packages in 1.15s
Prepared 1 package in 811ms
Uninstalled 1 package in 0.39ms
Installed 1 package in 0.81ms
 ~ xhuman==0.1.0 (from file:///content/XHUMAN)


In [4]:
from huggingface_hub import login
login()

In [4]:

from contextlib import nullcontext
from typing import Any

import torch
from accelerate import Accelerator
from accelerate.utils import DistributedDataParallelKwargs
from torch.optim import Optimizer

from lerobot.configs import parser
from lerobot.datasets.sampler import EpisodeAwareSampler
from lerobot.datasets.utils import cycle
from lerobot.optim.factory import make_optimizer_and_scheduler
from lerobot.policies.pretrained import PreTrainedPolicy
from lerobot.utils.logging_utils import AverageMeter, MetricsTracker
from lerobot.utils.random_utils import set_seed
from lerobot.utils.train_utils import load_training_state
from lerobot.utils.utils import (
    format_big_number,
    has_method,
    init_logging,
)

from xhuman.policies.factory import make_xhuman_policy, make_xhuman_pre_post_processors
from xhuman.configs.train import TrainPipelineConfigXHUMAN
from xhuman.datasets.factory import make_dataset_xhuman
from xhuman.datasets.utils import split_train_eval_episodes
from xhuman.logger import logger

## Helper Functions

These functions handle dataset loading and policy updates. They are designed to work with distributed training using HuggingFace Accelerate.

In [5]:
def load_dataset(cfg: TrainPipelineConfigXHUMAN, episodes: list[int], is_main_process: bool = True, accelerator: Accelerator | None = None):
    """
    Load the dataset for training and evaluation.
    """
    # Dataset loading synchronization: main process downloads first to avoid race conditions
    cfg.dataset.episodes = episodes

    if is_main_process:
        logger.info("Creating dataset")
        dataset = make_dataset_xhuman(cfg)

    accelerator.wait_for_everyone()

    # Now all other processes can safely load the dataset
    if not is_main_process:
        dataset = make_dataset_xhuman(cfg)

    return dataset

In [27]:
def update_policy(
    train_metrics: MetricsTracker,
    policy: PreTrainedPolicy,
    batch: Any,
    optimizer: Optimizer,
    grad_clip_norm: float,
    accelerator: Accelerator,
    lr_scheduler=None,
    lock=None,
) -> tuple[MetricsTracker, dict]:
    """
    Performs a single training step to update the policy's weights.

    This function executes the forward and backward passes,
    clips gradients, and steps the optimizer and
    learning rate scheduler. Accelerator handles
    mixed-precision training automatically.

    Args:
        train_metrics: A MetricsTracker instance to record training statistics.
        policy: The policy model to be trained.
        batch: A batch of training data.
        optimizer: The optimizer used to update the policy's parameters.
        grad_clip_norm: The maximum norm for gradient clipping.
        accelerator: The Accelerator instance for distributed training and mixed precision.
        lr_scheduler: An optional learning rate scheduler.
        lock: An optional lock for thread-safe optimizer updates.

    Returns:
        A tuple containing:
        - The updated MetricsTracker with new statistics for this step.
        - A dictionary of outputs from the policy's forward pass, for logging purposes.
    """
    policy.train()

    # Let accelerator handle mixed precision
    with accelerator.autocast():

        losses, dict_losses = policy.forward(batch)

        discrete_loss = losses[0]
        flow_matching_loss = losses[1]
        # subtask_loss = losses[2]
        ce_loss = discrete_loss  # subtask_loss is currently disabled

        total_loss = ce_loss + policy.config.alpha * flow_matching_loss
        # Sum per-sample losses to get scalar for backward pass
        total_loss_scalar = total_loss.mean()

    # Use accelerator's backward method
    accelerator.backward(total_loss_scalar)

    # Clip gradients if specified
    if grad_clip_norm > 0:
        grad_norm = accelerator.clip_grad_norm_(policy.parameters(), grad_clip_norm)
    else:
        grad_norm = torch.nn.utils.clip_grad_norm_(
            policy.parameters(), float("inf"), error_if_nonfinite=False
        )

    # Optimizer step
    with lock if lock is not None else nullcontext():
        optimizer.step()

    optimizer.zero_grad()

    # Step through pytorch scheduler at every batch instead of epoch
    if lr_scheduler is not None:
        lr_scheduler.step()

    # Update internal buffers if policy has update method
    if has_method(accelerator.unwrap_model(policy, keep_fp32_wrapper=True), "update"):
        accelerator.unwrap_model(policy, keep_fp32_wrapper=True).update()

    # Track losses for logging
    # total_loss is per-sample [B], so we track the mean
    train_metrics.loss = total_loss_scalar.item()
    train_metrics.grad_norm = grad_norm.item()
    train_metrics.lr = optimizer.param_groups[0]["lr"]

    # Create output dictionary with tracked losses
    output_dict = {
        "ce_loss": ce_loss.mean().item(),
        "flow_matching_loss": flow_matching_loss.mean().item(),
        "total_loss": total_loss_scalar.item(),
    }

    return train_metrics, output_dict

In [8]:
from xhuman.policies.pi05.configuration_pi05 import PI05Config

policy_config = PI05Config(repo_id="none",device="cuda")

## Configuration and Setup

Configure your dataset and policy settings here. The dataset configuration specifies which HuggingFace repository to load, and the policy configuration sets up the PI05 model architecture.

In [9]:
from xhuman.configs.default import LerobotDatasetConfig

dataset_config = LerobotDatasetConfig(
    repo_id="NONHUMAN-RESEARCH/pick-and-place-fruits-v2-test",
)

In [10]:
cfg = TrainPipelineConfigXHUMAN(
    dataset=dataset_config,
    policy=policy_config # Example policy configuration, replace with your actual policy path
)
cfg.validate()

## Training Setup

Initialize the Accelerator for distributed training and set up the training environment. The accelerator automatically handles:
- Multi-GPU training
- Mixed precision training
- Gradient synchronization across processes

In [11]:
# Create Accelerator
# It will automatically detect if running in distributed mode or single-process mode
# We set step_scheduler_with_optimizer=False to prevent accelerate from adjusting the lr_scheduler steps based on the num_processes
# We set find_unused_parameters=True to handle models with conditional computation
ddp_kwargs = DistributedDataParallelKwargs(find_unused_parameters=True)
accelerator = Accelerator(step_scheduler_with_optimizer=False, kwargs_handlers=[ddp_kwargs])

init_logging(accelerator=accelerator)

# Determine if this is the main process (for logging and checkpointing)
is_main_process = accelerator.is_main_process

# Set seed if specified
if cfg.seed is not None:
    set_seed(cfg.seed, accelerator=accelerator)

# Use accelerator's device
device = accelerator.device
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True

In [12]:
# ============================================================================
# Dataset Loading with Episode Filtering
# ============================================================================
# This cell loads the dataset with proper episode filtering.
# For debugging: Set DEBUG_MODE = True to use only a subset of episodes
# For production: Set DEBUG_MODE = False to use all available episodes

DEBUG_MODE = True  # Set to False for full training
DEBUG_MAX_EPISODES = 3  # Use only first N episodes for debugging

# First, get total episodes count (load minimal dataset to check)
if is_main_process:
    temp_dataset = make_dataset_xhuman(cfg)
    total_episodes = temp_dataset.meta.total_episodes
    del temp_dataset
    logger.info(f"Total episodes available: {total_episodes}")
else:
    # For non-main processes, use a reasonable default
    # In practice, this will be synced after main process loads
    total_episodes = 4  # Fallback - adjust if needed

accelerator.wait_for_everyone()

# Limit episodes for debugging
if DEBUG_MODE:
    episodes = list(range(min(DEBUG_MAX_EPISODES, total_episodes)))
    if is_main_process:
        logger.info(f"DEBUG MODE: Using only {len(episodes)} episodes")
else:
    episodes = list(range(total_episodes))

# Split episodes
train_episodes, eval_episodes = split_train_eval_episodes(
    episodes, split_ratio=cfg.split_ratio, seed=42
)

# Load dataset with ONLY train episodes (proper way to filter)
# This uses the load_dataset helper function which sets cfg.dataset.episodes
if is_main_process:
    logger.info(f"Loading train dataset with {len(train_episodes)} episodes")
dataset = load_dataset(cfg, train_episodes, is_main_process=is_main_process, accelerator=accelerator)

[15:12:47] INFO     Total episodes available: 11                                     ]8;id=370116;file:///tmp/ipython-input-1033998473.py\ipython-input-1033998473.py]8;;\:]8;id=65997;file:///tmp/ipython-input-1033998473.py#16\16]8;;\

           INFO     DEBUG MODE: Using only 3 episodes                                ]8;id=232594;file:///tmp/ipython-input-1033998473.py\ipython-input-1033998473.py]8;;\:]8;id=703849;file:///tmp/ipython-input-1033998473.py#28\28]8;;\

           INFO     Loading train dataset with 2 episodes                            ]8;id=146316;file:///tmp/ipython-input-1033998473.py\ipython-input-1033998473.py]8;;\:]8;id=772246;file:///tmp/ipython-input-1033998473.py#40\40]8;;\

           INFO     Creating dataset                                                  ]8;id=619176;file:///tmp/ipython-input-1989669429.py\ipython-input-1989669429.py]8;;\:]8;id=442417;file:///tmp/ipython-input-1989669429.py#9\9]8;;\

In [13]:
# Create policy
if is_main_process:
    logger.info("Creating policy")
policy = make_xhuman_policy(
    cfg=cfg.policy,
    ds_meta=dataset.meta,
)

accelerator.wait_for_everyone()

[15:12:50] INFO     Creating policy                                                   ]8;id=631262;file:///tmp/ipython-input-3499831122.py\ipython-input-3499831122.py]8;;\:]8;id=27824;file:///tmp/ipython-input-3499831122.py#3\3]8;;\

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` f

## Dataset and Model Information

Display metadata about the loaded dataset and model. This includes:
- Total number of episodes and frames
- Model parameter counts
- Effective batch size (accounting for distributed training)

In [14]:
# Display dataset metadata and model configuration
if is_main_process:
    from pprint import pprint

    print("=" * 80)
    print("DATASET METADATA")
    print("=" * 80)
    print(f"\nDataset Repository: {dataset.repo_id}")
    print(f"Total Episodes: {dataset.meta.total_episodes}")
    print(f"Training Episodes: {len(train_episodes)}")
    print(f"Number of Frames: {dataset.num_frames:,}")
    print(f"Number of Episodes (loaded): {dataset.num_episodes}")


    print("\n" + "=" * 80)

DATASET METADATA

Dataset Repository: NONHUMAN-RESEARCH/pick-and-place-fruits-v2-test
Total Episodes: 11
Training Episodes: 2
Number of Frames: 2,924
Number of Episodes (loaded): 2



In [15]:
processor_kwargs = {}
postprocessor_kwargs = {}

if (cfg.policy.pretrained_path and not cfg.resume) or not cfg.policy.pretrained_path:
    processor_kwargs["dataset_stats"] = dataset.meta.stats

if cfg.policy.pretrained_path is not None:
    processor_kwargs["preprocessor_overrides"] = {
        "device_processor": {"device": device.type},
        "normalizer_processor": {
            "stats": dataset.meta.stats,
            "features": {**policy.config.input_features, **policy.config.output_features},
            "norm_map": policy.config.normalization_mapping,
        },
    }
    postprocessor_kwargs["postprocessor_overrides"] = {
        "unnormalizer_processor": {
            "stats": dataset.meta.stats,
            "features": policy.config.output_features,
            "norm_map": policy.config.normalization_mapping,
        },
    }

In [ ]:
# This cell was removed - autoreload is not needed for production training

In [16]:
# Create processors
preprocessor, postprocessor = make_xhuman_pre_post_processors(
    policy_cfg=cfg.policy,
    pretrained_path=cfg.policy.pretrained_path,
    **processor_kwargs,
    **postprocessor_kwargs,
)

In [17]:
# Create optimizer and scheduler
if is_main_process:
    logger.info("Creating optimizer and scheduler")
optimizer, lr_scheduler = make_optimizer_and_scheduler(cfg, policy)

step = 0  # number of policy updates

# Resume from checkpoint if needed
if cfg.resume:
    step, optimizer, lr_scheduler = load_training_state(cfg.checkpoint_path, optimizer, lr_scheduler)

[15:14:10] INFO     Creating optimizer and scheduler                                  ]8;id=439898;file:///tmp/ipython-input-1795964688.py\ipython-input-1795964688.py]8;;\:]8;id=231148;file:///tmp/ipython-input-1795964688.py#3\3]8;;\

In [18]:
# Print training info
if is_main_process:
    num_learnable_params = sum(p.numel() for p in policy.parameters() if p.requires_grad)
    num_total_params = sum(p.numel() for p in policy.parameters())
    logger.info(f"Output dir: {cfg.output_dir}")
    logger.info(f"Steps: {cfg.steps} ({format_big_number(cfg.steps)})")
    logger.info(f"Dataset frames: {dataset.num_frames} ({format_big_number(dataset.num_frames)})")
    logger.info(f"Dataset episodes: {dataset.num_episodes}")
    num_processes = accelerator.num_processes
    effective_bs = cfg.batch_size * num_processes
    logger.info(f"Effective batch size: {cfg.batch_size} x {num_processes} = {effective_bs}")
    logger.info(f"Learnable params: {num_learnable_params} ({format_big_number(num_learnable_params)})")
    logger.info(f"Total params: {num_total_params} ({format_big_number(num_total_params)})")

           INFO     Output dir: outputs/train/2026-01-14/15-12-44_pi05_ki             ]8;id=795667;file:///tmp/ipython-input-4288334519.py\ipython-input-4288334519.py]8;;\:]8;id=844962;file:///tmp/ipython-input-4288334519.py#5\5]8;;\

           INFO     Steps: 100000 (100K)                                              ]8;id=163032;file:///tmp/ipython-input-4288334519.py\ipython-input-4288334519.py]8;;\:]8;id=225772;file:///tmp/ipython-input-4288334519.py#6\6]8;;\

           INFO     Dataset frames: 2924 (3K)                                         ]8;id=101414;file:///tmp/ipython-input-4288334519.py\ipython-input-4288334519.py]8;;\:]8;id=376417;file:///tmp/ipython-input-4288334519.py#7\7]8;;\

           INFO     Dataset episodes: 2                                               ]8;id=45561;file:///tmp/ipython-input-4288334519.py\ipython-input-4288334519.py]8;;\:]8;id=765179;file:///tmp/ipython-input-4288334519.py#8\8]8;;\

           INFO     Effective batch size: 8 x 1 = 8                                  ]8;id=82627;file:///tmp/ipython-input-4288334519.py\ipython-input-4288334519.py]8;;\:]8;id=578856;file:///tmp/ipython-input-4288334519.py#11\11]8;;\

           INFO     Learnable params: 3616757520 (4B)                                ]8;id=903565;file:///tmp/ipython-input-4288334519.py\ipython-input-4288334519.py]8;;\:]8;id=379201;file:///tmp/ipython-input-4288334519.py#12\12]8;;\

           INFO     Total params: 3616757520 (4B)                                    ]8;id=693384;file:///tmp/ipython-input-4288334519.py\ipython-input-4288334519.py]8;;\:]8;id=238968;file:///tmp/ipython-input-4288334519.py#13\13]8;;\

In [19]:
# Create dataloader
if hasattr(cfg.policy, "drop_n_last_frames"):
    logger.info(f"Dropping {cfg.policy.drop_n_last_frames} last frames")
    shuffle = False
    sampler = EpisodeAwareSampler(
        dataset.meta.episodes["dataset_from_index"],
        dataset.meta.episodes["dataset_to_index"],
        drop_n_last_frames=cfg.policy.drop_n_last_frames,
        shuffle=True,
    )
else:
    logger.info("Not dropping any frames")
    shuffle = True
    sampler = None



[15:14:11] INFO     Not dropping any frames                                          ]8;id=105907;file:///tmp/ipython-input-4111286182.py\ipython-input-4111286182.py]8;;\:]8;id=398591;file:///tmp/ipython-input-4111286182.py#12\12]8;;\

In [20]:
from torch.utils.data import Dataset

class SmartSubset(Dataset):
    def __init__(self, dataset, indices):
        self.dataset = dataset
        self.indices = indices

    def __getitem__(self, idx):
        return self.dataset[self.indices[idx]]

    def __len__(self):
        return len(self.indices)

    def __getattr__(self, name):
        # This is the magic part:
        # If the code asks for 'meta', 'fps', etc., and this class doesn't have it,
        # it automatically looks inside the original dataset.
        return getattr(self.dataset, name)

# --- USAGE ---
# Use SmartSubset instead of torch.utils.data.Subset
debug_subset = SmartSubset(dataset, range(0, 50))

# Now create your loader normally
train_dataloader = torch.utils.data.DataLoader(
    debug_subset,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

In [21]:
# Prepare everything with accelerator
accelerator.wait_for_everyone()
policy, optimizer, dataloader, lr_scheduler = accelerator.prepare(
    policy, optimizer, train_dataloader, lr_scheduler
)

policy.train()

# Setup metrics tracking
train_metrics = {
    "loss": AverageMeter("loss", ":.3f"),
    "grad_norm": AverageMeter("grdn", ":.3f"),
    "lr": AverageMeter("lr", ":0.1e"),
    "update_s": AverageMeter("updt_s", ":.3f"),
    "dataloading_s": AverageMeter("data_s", ":.3f"),
}

effective_batch_size = cfg.batch_size * accelerator.num_processes

In [22]:
train_tracker = MetricsTracker(
    effective_batch_size,
    dataset.num_frames,
    dataset.num_episodes,
    train_metrics,
    initial_step=step,
    accelerator=accelerator,
)

In [23]:
"""
dataloader = torch.utils.data.DataLoader(
    dataset,
    num_workers=cfg.num_workers,
    batch_size=2,
    shuffle=shuffle and not cfg.dataset.streaming,
    sampler=sampler,
    pin_memory=device.type == "cuda",
    drop_last=False,
    prefetch_factor=2 if cfg.num_workers > 0 else None,
)
"""

'\ndataloader = torch.utils.data.DataLoader(\n    dataset,\n    num_workers=cfg.num_workers,\n    batch_size=2,\n    shuffle=shuffle and not cfg.dataset.streaming,\n    sampler=sampler,\n    pin_memory=device.type == "cuda",\n    drop_last=False,\n    prefetch_factor=2 if cfg.num_workers > 0 else None,\n)\n'

In [24]:
# Training initialization
# This logs the start of training and shows how many episodes will be used
if is_main_process:
    logger.info("Start offline training on a fixed dataset")
    logger.info(f"Train episodes: {len(train_episodes)}")
    logger.info(f"Total training steps: {cfg.steps}")

[15:14:18] INFO     Start offline training on a fixed dataset                         ]8;id=388162;file:///tmp/ipython-input-2557974599.py\ipython-input-2557974599.py]8;;\:]8;id=372528;file:///tmp/ipython-input-2557974599.py#4\4]8;;\

           INFO     Train episodes: 2                                                 ]8;id=716751;file:///tmp/ipython-input-2557974599.py\ipython-input-2557974599.py]8;;\:]8;id=679514;file:///tmp/ipython-input-2557974599.py#5\5]8;;\

           INFO     Total training steps: 100000                                      ]8;id=764544;file:///tmp/ipython-input-2557974599.py\ipython-input-2557974599.py]8;;\:]8;id=256702;file:///tmp/ipython-input-2557974599.py#6\6]8;;\

In [25]:

dl_iter = cycle(dataloader)

In [31]:
batch["observation.images.right"].shape

torch.Size([2, 3, 376, 672])

In [35]:
batch["action"].shape

torch.Size([2, 50, 14])

In [32]:
train_tracker

In [28]:
for step in range(0, 2): # Or however many steps you want
    # --- SAFE ITERATOR LOGIC ---
    batch = next(dl_iter)
    batch = preprocessor(batch)

    # print(batch)
    # Correcting the argument: pass the 'train_tracker' instance, not the 'MetricsTracker' class
    train_tracker, output_dict = update_policy(
        train_tracker,
        policy,
        batch,
        optimizer,
        cfg.optimizer.grad_clip_norm,
        accelerator=accelerator,
        lr_scheduler=lr_scheduler,
    )
    print(train_tracker)
    print(output_dict)

step:0 smpl:0 ep:0 epch:0.00 loss:30.665 grdn:202.285 lr:6.2e-08 updt_s:0.000 data_s:0.000
{'ce_loss': 14.278895378112793, 'flow_matching_loss': 1.6530702114105225, 'total_loss': 30.80959701538086}
step:0 smpl:0 ep:0 epch:0.00 loss:30.811 grdn:208.508 lr:7.5e-08 updt_s:0.000 data_s:0.000
{'ce_loss': 14.0741548538208, 'flow_matching_loss': 1.7029062509536743, 'total_loss': 31.10321807861328}
